In [8]:
ord('b')-ord('a')

1

In [1]:
from bertopic import BERTopic
from transformers import AutoTokenizer, AutoModel, pipeline
import pandas as pd
import pickle
import torch
import numpy as np
from bertopic.representation import KeyBERTInspired
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from bertopic.vectorizers import ClassTfidfTransformer
import openpyxl
import re

2023-11-04 14:36:04.972788: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-04 14:36:16.133663: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-11-04 14:36:16.134248: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-11-04 14:36:16.134308: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
df = pd.read_csv(
    "/home/maaz-lfd/Maaz/Thesis/Thesis/dataset/clinical_trials/round_5_CT_metdata.csv"
)
df

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id
0,5yhe786e,3bb07ea10432f7738413dff9816809cc90f03f99,PMC,Debate: Transfusing to normal haemoglobin leve...,10.1186/cc987,PMC137267,11299062.0,no-cc,Recent evidence suggests that critically ill p...,2001-03-08,"Alvarez, Gonzalo; Hébert, Paul C; Szick, Sharyn",Crit Care,NaN,NaN,NaN,document_parses/pdf_json/3bb07ea10432f7738413d...,document_parses/pmc_json/PMC137267.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN
1,8zchiykl,5806726a24dc91de3954001effbdffd7a82d54e2,PMC,The 21st International Symposium on Intensive ...,10.1186/cc1013,PMC137274,11353930.0,no-cc,The 21st International Symposium on Intensive ...,2001-05-02,"Ball, Jonathan; Venn, Richard",Crit Care,NaN,NaN,NaN,document_parses/pdf_json/5806726a24dc91de39540...,document_parses/pmc_json/PMC137274.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN
2,x7wg7e9s,e7f311d3c752063f75f28de85abd5f82eacab413,PMC,Epicatechins Purified from Green Tea (Camellia...,10.1093/ecam/nel003,PMC1475929,16786054.0,no-cc,The anticancer potential of catechins derived ...,2006-04-25,"Ravindranath, Mepur H.; Saravanan, Thiruverkad...",Evid Based Complement Alternat Med,NaN,NaN,NaN,document_parses/pdf_json/e7f311d3c752063f75f28...,document_parses/pmc_json/PMC1475929.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN
3,utglk4af,babad3896fca8ed7c89d9b785c08e7c42a48a6a2,PMC,Markers of exacerbation severity in chronic ob...,10.1186/1465-9921-7-74,PMC1481583,16686949.0,cc-by,BACKGROUND: Patients with chronic obstructive ...,2006-05-10,"Franciosi, Luigi G; Page, Clive P; Celli, Bart...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/babad3896fca8ed7c89d9...,document_parses/pmc_json/PMC1481583.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN
4,hwlvk68z,0dcf7336a3569d4103dc5be47ddd6b583dea98f0,PMC,Transmission patterns of smallpox: systematic ...,10.1186/1471-2458-6-126,PMC1525174,16677388.0,cc-by,BACKGROUND: Because smallpox (variola major) m...,2006-05-05,"Bhatnagar, Vibha; Stoto, Michael A; Morton, Sa...",BMC Public Health,NaN,NaN,NaN,document_parses/pdf_json/0dcf7336a3569d4103dc5...,document_parses/pmc_json/PMC1525174.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3627,i1il6jjs,5a5a25d733b227ba661f3ba8f69af5886999f02c,Medline; PMC,Human cytomegalovirus and Herpes Simplex type ...,10.18632/oncotarget.22106,PMC5722503,29228551.0,cc-by,Human cytomegalovirus (HCMV) utilizes RNA poly...,2017-10-29,"Kostopoulou, Ourania N.; Wilhelmi, Vanessa; Ra...",Oncotarget,NaN,NaN,NaN,document_parses/pdf_json/5a5a25d733b227ba661f3...,document_parses/pmc_json/PMC5722503.xml.json,https://doi.org/10.18632/oncotarget.22106; htt...,30028157.0
3628,bw6a5gmy,e8c436a170cb0a1b7024effb168bb8a7214f2021,Medline; PMC,Pre-existing immunity to SARS-CoV-2: the known...,10.1038/s41577-020-0389-z,PMC7339790,32636479.0,no-cc,T cell reactivity against SARS-CoV-2 was obser...,2020-07-07,"Sette, Alessandro; Crotty, Shane",Nat Rev Immunol,NaN,NaN,NaN,document_parses/pdf_json/e8c436a170cb0a1b7024e...,document_parses/pmc_json/PMC7339790.xml.json,https://doi.org/10.1038/s41577-020-0389-z; htt...,220381745.0
3629,rpwccvqi,96abc8769d5840bdeac8590d795561706f34b4c4,Medline; PMC,Core Minimal Datasets to Advance Clinical Rese...,10.1093/cid/ciz760,PMC7108131,31406989.0,cc-by,The Ebola virus disease outbreak in west Afric...,2020-02-15,"Rojek, Amanda M; Moran, James; Horby, Peter W",Clin Infect Dis,NaN,NaN,NaN,document_parses/pdf_json/96abc8769d5840bdeac85...,document_parses/pmc_json/PMC7108131.xml.json,https://doi.org/10.1093/cid/ciz760; https://ww...,199549631.0
3630,9th2jiq6,97229e8af52b75aa9ac0f0eb530bf89f7f5ec134,Medline; PMC,Cyclophilin A Associates with Enterovirus-71 V...,10.1371/journal.ppat.1004422,PMC4183573,25275585.0,cc-by,Viruses

#### Remove PMCIDs that are already categorized

In [3]:
# Replace 'your_file.xlsx' with the path to your Excel file
file_path = '/home/maaz-lfd/Maaz/Thesis/Thesis/dataset/clinical_trials/Topic_Categories.xlsx'
pmcids = []
pattern = r'PMC\d*'

# Load the Excel file
workbook = openpyxl.load_workbook(file_path)

# Get the active sheet (you can also specify a specific sheet by name)
sheet = workbook.active

# Iterate through all the cells in the sheet and check for comments
for row in sheet.iter_rows():
    for cell in row:
        comment = cell.comment
        if comment:
            pmcids.extend(re.findall(pattern, comment.text))

# Close the Excel file
workbook.close()
print(pmcids)

['PMC7033907', 'PMC7123788', 'PMC7286676', 'PMC7033907', 'PMC7121048', 'PMC7059709', 'PMC7351640', 'PMC7226903', 'PMC3951450', 'PMC4709057', 'PMC5755876', 'PMC3422077', 'PMC6156540', 'PMC7103727', 'PMC2768779', 'PMC4494249', 'PMC7296525', 'PMC3630917', 'PMC2702340', 'PMC7222935', 'PMC7154102', 'PMC7252419', 'PMC7248264', 'PMC7121035', 'PMC7222980', 'PMC7033237', 'PMC4963060', 'PMC7114313', 'PMC2878469', 'PMC7223555', 'PMC7121119', 'PMC7089977', 'PMC7088979', 'PMC7167690', 'PMC7114996', 'PMC7122597', 'PMC7109106', 'PMC7121101', 'PMC7123330', 'PMC7112877', 'PMC7087648', 'PMC5280084', 'PMC7121977', 'PMC7359443', 'PMC7120928', 'PMC7314365', 'PMC3488323', 'PMC4169708', 'PMC7120914', 'PMC7159066', 'PMC7223541', 'PMC7121977', 'PMC7262099', 'PMC7314365', 'PMC6722851', 'PMC3135369', 'PMC7168445', 'PMC5217322', 'PMC7224078', 'PMC7304253', 'PMC7348566', 'PMC6613463', 'PMC7100215', 'PMC5774686', 'PMC7100576', 'PMC7108198', 'PMC3878835', 'PMC6404786', 'PMC7100484', 'PMC4383725', 'PMC5796004', 'PMC3

In [36]:
len(set(pmcids))

519

In [5]:
df = df[~df['pmcid'].isin(set(pmcids))]
df

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id
0,5yhe786e,3bb07ea10432f7738413dff9816809cc90f03f99,PMC,Debate: Transfusing to normal haemoglobin leve...,10.1186/cc987,PMC137267,11299062.0,no-cc,Recent evidence suggests that critically ill p...,2001-03-08,"Alvarez, Gonzalo; Hébert, Paul C; Szick, Sharyn",Crit Care,NaN,NaN,NaN,document_parses/pdf_json/3bb07ea10432f7738413d...,document_parses/pmc_json/PMC137267.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN
2,x7wg7e9s,e7f311d3c752063f75f28de85abd5f82eacab413,PMC,Epicatechins Purified from Green Tea (Camellia...,10.1093/ecam/nel003,PMC1475929,16786054.0,no-cc,The anticancer potential of catechins derived ...,2006-04-25,"Ravindranath, Mepur H.; Saravanan, Thiruverkad...",Evid Based Complement Alternat Med,NaN,NaN,NaN,document_parses/pdf_json/e7f311d3c752063f75f28...,document_parses/pmc_json/PMC1475929.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN
3,utglk4af,babad3896fca8ed7c89d9b785c08e7c42a48a6a2,PMC,Markers of exacerbation severity in chronic ob...,10.1186/1465-9921-7-74,PMC1481583,16686949.0,cc-by,BACKGROUND: Patients with chronic obstructive ...,2006-05-10,"Franciosi, Luigi G; Page, Clive P; Celli, Bart...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/babad3896fca8ed7c89d9...,document_parses/pmc_json/PMC1481583.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN
4,hwlvk68z,0dcf7336a3569d4103dc5be47ddd6b583dea98f0,PMC,Transmission patterns of smallpox: systematic ...,10.1186/1471-2458-6-126,PMC1525174,16677388.0,cc-by,BACKGROUND: Because smallpox (variola major) m...,2006-05-05,"Bhatnagar, Vibha; Stoto, Michael A; Morton, Sa...",BMC Public Health,NaN,NaN,NaN,document_parses/pdf_json/0dcf7336a3569d4103dc5...,document_parses/pmc_json/PMC1525174.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN
6,nzh87aux,9b7a0ad7b6c7f59e7a6cf1dc9d07912a273d19b5,PMC,The Waiting Time for Inter-Country Spread of P...,10.1371/journal.pone.0000143,PMC1764036,17206278.0,cc-by,BACKGROUND: The time delay between the start o...,2007-01-03,"Caley, Peter; Becker, Niels G.; Philp, David J.",PLoS One,NaN,NaN,NaN,document_parses/pdf_json/9b7a0ad7b6c7f59e7a6cf...,document_parses/pmc_json/PMC1764036.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3627,i1il6jjs,5a5a25d733b227ba661f3ba8f69af5886999f02c,Medline; PMC,Human cytomegalovirus and Herpes Simplex type ...,10.18632/oncotarget.22106,PMC5722503,29228551.0,cc-by,Human cytomegalovirus (HCMV) utilizes RNA poly...,2017-10-29,"Kostopoulou, Ourania N.; Wilhelmi, Vanessa; Ra...",Oncotarget,NaN,NaN,NaN,document_parses/pdf_json/5a5a25d733b227ba661f3...,document_parses/pmc_json/PMC5722503.xml.json,https://doi.org/10.18632/oncotarget.22106; htt...,30028157.0
3628,bw6a5gmy,e8c436a170cb0a1b7024effb168bb8a7214f2021,Medline; PMC,Pre-existing immunity to SARS-CoV-2: the known...,10.1038/s41577-020-0389-z,PMC7339790,32636479.0,no-cc,T cell reactivity against SARS-CoV-2 was obser...,2020-07-07,"Sette, Alessandro; Crotty, Shane",Nat Rev Immunol,NaN,NaN,NaN,document_parses/pdf_json/e8c436a170cb0a1b7024e...,document_parses/pmc_json/PMC7339790.xml.json,https://doi.org/10.1038/s41577-020-0389-z; htt...,220381745.0
3629,rpwccvqi,96abc8769d5840bdeac8590d795561706f34b4c4,Medline; PMC,Core Minimal Datasets to Advance Clinical Rese...,10.1093/cid/ciz760,PMC7108131,31406989.0,cc-by,The Ebola virus disease outbreak in west Afric...,2020-02-15,"Rojek, Amanda M; Moran, James; Horby, Peter W",Clin Infect Dis,NaN,NaN,NaN,document_parses/pdf_json/96abc8769d5840bdeac85...,document_parses/pmc_json/PMC7108131.xml.json,https://doi.org/10.1093/cid/ciz760; https://ww...,199549631.0
3630,9th2jiq6,97229e8af52b75aa9ac0f0eb530bf89f7f5ec134,Medline; PMC,Cyclophilin A Associates with Enterovirus-71 V...,10.1371/journal.ppat.1004422,PM

In [6]:
df.reset_index(inplace=True, drop=True)
df

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id
0,5yhe786e,3bb07ea10432f7738413dff9816809cc90f03f99,PMC,Debate: Transfusing to normal haemoglobin leve...,10.1186/cc987,PMC137267,11299062.0,no-cc,Recent evidence suggests that critically ill p...,2001-03-08,"Alvarez, Gonzalo; Hébert, Paul C; Szick, Sharyn",Crit Care,NaN,NaN,NaN,document_parses/pdf_json/3bb07ea10432f7738413d...,document_parses/pmc_json/PMC137267.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN
1,x7wg7e9s,e7f311d3c752063f75f28de85abd5f82eacab413,PMC,Epicatechins Purified from Green Tea (Camellia...,10.1093/ecam/nel003,PMC1475929,16786054.0,no-cc,The anticancer potential of catechins derived ...,2006-04-25,"Ravindranath, Mepur H.; Saravanan, Thiruverkad...",Evid Based Complement Alternat Med,NaN,NaN,NaN,document_parses/pdf_json/e7f311d3c752063f75f28...,document_parses/pmc_json/PMC1475929.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN
2,utglk4af,babad3896fca8ed7c89d9b785c08e7c42a48a6a2,PMC,Markers of exacerbation severity in chronic ob...,10.1186/1465-9921-7-74,PMC1481583,16686949.0,cc-by,BACKGROUND: Patients with chronic obstructive ...,2006-05-10,"Franciosi, Luigi G; Page, Clive P; Celli, Bart...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/babad3896fca8ed7c89d9...,document_parses/pmc_json/PMC1481583.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN
3,hwlvk68z,0dcf7336a3569d4103dc5be47ddd6b583dea98f0,PMC,Transmission patterns of smallpox: systematic ...,10.1186/1471-2458-6-126,PMC1525174,16677388.0,cc-by,BACKGROUND: Because smallpox (variola major) m...,2006-05-05,"Bhatnagar, Vibha; Stoto, Michael A; Morton, Sa...",BMC Public Health,NaN,NaN,NaN,document_parses/pdf_json/0dcf7336a3569d4103dc5...,document_parses/pmc_json/PMC1525174.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN
4,nzh87aux,9b7a0ad7b6c7f59e7a6cf1dc9d07912a273d19b5,PMC,The Waiting Time for Inter-Country Spread of P...,10.1371/journal.pone.0000143,PMC1764036,17206278.0,cc-by,BACKGROUND: The time delay between the start o...,2007-01-03,"Caley, Peter; Becker, Niels G.; Philp, David J.",PLoS One,NaN,NaN,NaN,document_parses/pdf_json/9b7a0ad7b6c7f59e7a6cf...,document_parses/pmc_json/PMC1764036.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3108,i1il6jjs,5a5a25d733b227ba661f3ba8f69af5886999f02c,Medline; PMC,Human cytomegalovirus and Herpes Simplex type ...,10.18632/oncotarget.22106,PMC5722503,29228551.0,cc-by,Human cytomegalovirus (HCMV) utilizes RNA poly...,2017-10-29,"Kostopoulou, Ourania N.; Wilhelmi, Vanessa; Ra...",Oncotarget,NaN,NaN,NaN,document_parses/pdf_json/5a5a25d733b227ba661f3...,document_parses/pmc_json/PMC5722503.xml.json,https://doi.org/10.18632/oncotarget.22106; htt...,30028157.0
3109,bw6a5gmy,e8c436a170cb0a1b7024effb168bb8a7214f2021,Medline; PMC,Pre-existing immunity to SARS-CoV-2: the known...,10.1038/s41577-020-0389-z,PMC7339790,32636479.0,no-cc,T cell reactivity against SARS-CoV-2 was obser...,2020-07-07,"Sette, Alessandro; Crotty, Shane",Nat Rev Immunol,NaN,NaN,NaN,document_parses/pdf_json/e8c436a170cb0a1b7024e...,document_parses/pmc_json/PMC7339790.xml.json,https://doi.org/10.1038/s41577-020-0389-z; htt...,220381745.0
3110,rpwccvqi,96abc8769d5840bdeac8590d795561706f34b4c4,Medline; PMC,Core Minimal Datasets to Advance Clinical Rese...,10.1093/cid/ciz760,PMC7108131,31406989.0,cc-by,The Ebola virus disease outbreak in west Afric...,2020-02-15,"Rojek, Amanda M; Moran, James; Horby, Peter W",Clin Infect Dis,NaN,NaN,NaN,document_parses/pdf_json/96abc8769d5840bdeac85...,document_parses/pmc_json/PMC7108131.xml.json,https://doi.org/10.1093/cid/ciz760; https://ww...,199549631.0
3111,9th2jiq6,97229e8af52b75aa9ac0f0eb530bf89f7f5ec134,Medline; PMC,Cyclophilin A Associates with Enterovirus-71 V...,10.1371/journal.ppat.1004422,PM

#### ....

Load **ClinicalBert** Model

In [7]:
tokenizer = AutoTokenizer.from_pretrained("medicalai/ClinicalBERT")
model = AutoModel.from_pretrained("medicalai/ClinicalBERT")

In [8]:
torch.cuda.is_available()

True

In [9]:
# Set CUDA device if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cuda")
device

device(type='cuda')

In [10]:
model.to(device)

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0): TransformerBlock(
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): Linear(

In [11]:
model.eval()
first_embedding = True
for i in range(len(df)):
    print("count ", i + 1)
    tokens = tokenizer(
        df["abstract"][i],
        padding="max_length",
        truncation=True,
        max_length=512,
        return_tensors="pt",
    )
    tokens = {key: val.to(device) for key, val in tokens.items()}
    with torch.no_grad():
        outputs = model(**tokens)
        embedding = outputs.last_hidden_state.mean(dim=1).cpu().numpy()
        if first_embedding == True:
            with open(
                "/home/maaz-lfd/Maaz/Thesis/Thesis/dataset/clinical_trials/slice_abstract_embedding_clinicalBERT_3113",
                "wb",
            ) as f:
                pickle.dump(embedding, f)

            first_embedding = False
        else:
            saved_emb = pickle.loads(
                open(
                    "/home/maaz-lfd/Maaz/Thesis/Thesis/dataset/clinical_trials/slice_abstract_embedding_clinicalBERT_3113",
                    "rb",
                ).read()
            )
            concat_emb = np.concatenate((embedding, saved_emb))
            print("shape after concatination ", concat_emb.shape)
            with open(
                "/home/maaz-lfd/Maaz/Thesis/Thesis/dataset/clinical_trials/slice_abstract_embedding_clinicalBERT_3113",
                "wb",
            ) as f:
                pickle.dump(concat_emb, f)

count  1
count  2
shape after concatination  (2, 768)
count  3
shape after concatination  (3, 768)
count  4
shape after concatination  (4, 768)
count  5
shape after concatination  (5, 768)
count  6
shape after concatination  (6, 768)
count  7
shape after concatination  (7, 768)
count  8
shape after concatination  (8, 768)
count  9
shape after concatination  (9, 768)
count  10
shape after concatination  (10, 768)
count  11
shape after concatination  (11, 768)
count  12
shape after concatination  (12, 768)
count  13
shape after concatination  (13, 768)
count  14
shape after concatination  (14, 768)
count  15
shape after concatination  (15, 768)
count  16
shape after concatination  (16, 768)
count  17
shape after concatination  (17, 768)
count  18
shape after concatination  (18, 768)
count  19
shape after concatination  (19, 768)
count  20
shape after concatination  (20, 768)
count  21
shape after concatination  (21, 768)
count  22
shape after concatination  (22, 768)
count  23
shape afte

In [12]:
concat_emb.shape

(3113, 768)

In [13]:
representation_model = KeyBERTInspired()

# Step 2 - Reduce dimensionality
umap_model = UMAP(n_neighbors=50, n_components=200, min_dist=0.0, metric="cosine", random_state=42)

# Step 3 - Cluster reduced embeddings
hdbscan_model = HDBSCAN(min_cluster_size=10, metric="euclidean", cluster_selection_method="eom", prediction_data=True)

# Step 4 - Tokenize topics
vectorizer_model = CountVectorizer(stop_words="english", min_df=2, ngram_range=(1, 2))
# vectorizer_model = TfidfVectorizer(stop_words="english", min_df=2, ngram_range=(1, 2))

# Step 5 - Create topic representation
ctfidf_model = ClassTfidfTransformer()

# Step 6 - (Optional) Fine-tune topic representations with 
# a `bertopic.representation` model
representation_model = KeyBERTInspired()

topic_model = BERTopic(umap_model=umap_model, hdbscan_model=hdbscan_model, vectorizer_model=vectorizer_model,  ctfidf_model=ctfidf_model 
                       ,top_n_words=10, verbose=True)

#### TOPIC MODELING USING CLINICAL BERT

In [14]:
clinical_bert_embeddings = pickle.loads(  open("/home/maaz-lfd/Maaz/Thesis/Thesis/dataset/clinical_trials/slice_abstract_embedding_clinicalBERT_3113","rb",).read() )

In [15]:
topics, probs = topic_model.fit_transform(df["abstract"].to_list(), clinical_bert_embeddings)

2023-11-04 14:50:24,605 - BERTopic - Reduced dimensionality
2023-11-04 14:50:27,809 - BERTopic - Clustered reduced embeddings


In [16]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,2027,-1_patients_covid19_disease_clinical,"[patients, covid19, disease, clinical, treatme...",[BACKGROUND: Respiratory viruses are associate...
1,0,111,0_patients_clinical_covid19_treatment,"[patients, clinical, covid19, treatment, infec...",[OBJECTIVES: To investigate a nosocomial outbr...
2,1,102,1_patients_clinical_disease_covid19,"[patients, clinical, disease, covid19, review,...",[BACKGROUND: Despite increased efforts to cont...
3,2,98,2_patients_disease_covid19_virus,"[patients, disease, covid19, virus, respirator...",[BACKGROUND: The SARS outbreak of 2002–2003 pr...
4,3,95,3_patients_respiratory_infection_viral,"[patients, respiratory, infection, viral, covi...",[The current pandemic due to the severe acute ...
5,4,95,4_covid19_clinical_patients_cells,"[covid19, clinical, patients, cells, viral, di...",[BACKGROUND: Major infectious disease outbreak...
6,5,93,5_patients_infection_viral_respiratory,"[patients, infection, viral, respiratory, new,...",[OBJECTIVE: Acute exacerbations of chronic obs...
7,6,89,6_patients_covid19_respiratory_infection,"[patients, covid19, respiratory, infection, in...",[BACKGROUND: Southeast Asia has been at the ep...
8,7,53,7_cells_covid19_patients_cell,"[cells, covid19, patients, cell, surfactant, p...",[BACKGROUND: Pediatric ARDS still represents a...
9,8,38,8_tcm_acute_potential_covid19,"[tcm, acute, potential, covid19, derivatives, ...",[OBJECTIVE: Fever is a very common adaptive im...


In [17]:
topic_model.get_topic_info()['Representation'][1]

['patients',
 'clinical',
 'covid19',
 'treatment',
 'infection',
 'respiratory',
 'virus',
 'results',
 'cells',
 'disease']

In [18]:
# Intertopic Distance Map
topic_model.visualize_topics(title='<b>CLINICAL BERT</b>')

In [19]:
# Hierarchical Clustering
topic_model.visualize_hierarchy(title="<b>CLINICAL BERT</b>")

In [20]:
abs_topics = pd.DataFrame({"topic": topics, "pmcid":df['pmcid'], "abstracts": df["abstract"].to_list()})

In [21]:
abs_topics

,topic,pmcid,abstracts
0,3,PMC137267,Recent evidence suggests that critically ill p...
1,-1,PMC1475929,The anticancer potential of catechins derived ...
2,-1,PMC1481583,BACKGROUND: Patients with chronic obstructive ...
3,-1,PMC1525174,BACKGROUND: Because smallpox (variola major) m...
4,-1,PMC1764036,BACKGROUND: The time delay between the start o...
...,...,...,...
3108,2,PMC5722503,Human cytomegalovirus (HCMV) utilizes RNA poly...
3109,-1,PMC7339790,T cell reactivity against SARS-CoV-2 was obser...
3110,2,PMC7108131,The Ebola virus disease outbreak in west Afric...
3111,1,PMC4183573,Viruses utilize host factors for their efficie...


In [22]:
abs_topics.to_excel(
    "/home/maaz-lfd/Maaz/Thesis/Thesis/dataset/clinical_trials/topic_abstract_map.xlsx",
    index=False,
)

Load **PubMedELECTRA** model

In [23]:
tokenizer = AutoTokenizer.from_pretrained(
    "microsoft/BiomedNLP-PubMedELECTRA-large-uncased-abstract"  
)
model = AutoModel.from_pretrained(
    "microsoft/BiomedNLP-PubMedELECTRA-large-uncased-abstract"
)
# model = pipeline(model= "microsoft/BiomedNLP-PubMedELECTRA-large-uncased-abstract")

Some weights of the model checkpoint at microsoft/BiomedNLP-PubMedELECTRA-large-uncased-abstract were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [24]:
model.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 1024, padding_idx=0)
    (position_embeddings): Embedding(512, 1024)
    (token_type_embeddings): Embedding(2, 1024)
    (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=Fals

In [25]:
model.eval()
first_embedding = True
for i in range(len(df)):
    print("count ", i + 1)
    tokens = tokenizer(
        df["abstract"][i],
        padding="max_length",
        truncation=True,
        max_length=512,
        return_tensors="pt",
    )
    tokens = {key: val.to(device) for key, val in tokens.items()}
    with torch.no_grad():
        outputs = model(**tokens)
        embedding = outputs.last_hidden_state.mean(dim=1).cpu().numpy()
        if first_embedding == True:
            with open(
                "/home/maaz-lfd/Maaz/Thesis/Thesis/dataset/clinical_trials/slice_abstract_embedding_PubMedELECTRA_3113",
                "wb",
            ) as f:
                pickle.dump(embedding, f)

            first_embedding = False
        else:
            saved_emb = pickle.loads(
                open(
                    "/home/maaz-lfd/Maaz/Thesis/Thesis/dataset/clinical_trials/slice_abstract_embedding_PubMedELECTRA_3113",
                    "rb",
                ).read()
            )
            concat_emb = np.concatenate((embedding, saved_emb))
            print("shape after concatination ", concat_emb.shape)
            with open(
                "/home/maaz-lfd/Maaz/Thesis/Thesis/dataset/clinical_trials/slice_abstract_embedding_PubMedELECTRA_3113",
                "wb",
            ) as f:
                pickle.dump(concat_emb, f)

count  1
count  2
shape after concatination  (2, 1024)
count  3
shape after concatination  (3, 1024)
count  4
shape after concatination  (4, 1024)
count  5
shape after concatination  (5, 1024)
count  6
shape after concatination  (6, 1024)
count  7
shape after concatination  (7, 1024)
count  8
shape after concatination  (8, 1024)
count  9
shape after concatination  (9, 1024)
count  10
shape after concatination  (10, 1024)
count  11
shape after concatination  (11, 1024)
count  12
shape after concatination  (12, 1024)
count  13
shape after concatination  (13, 1024)
count  14
shape after concatination  (14, 1024)
count  15
shape after concatination  (15, 1024)
count  16
shape after concatination  (16, 1024)
count  17
shape after concatination  (17, 1024)
count  18
shape after concatination  (18, 1024)
count  19
shape after concatination  (19, 1024)
count  20
shape after concatination  (20, 1024)
count  21
shape after concatination  (21, 1024)
count  22
shape after concatination  (22, 1024)

#### TOPIC MODELING USING PUBMED ELECTRA

In [26]:
pubmed_electra_embeddings = pickle.loads(  open("/home/maaz-lfd/Maaz/Thesis/Thesis/dataset/clinical_trials/slice_abstract_embedding_PubMedELECTRA_3113","rb",).read() )

In [27]:
topics, probs = topic_model.fit_transform(df["abstract"].to_list(), pubmed_electra_embeddings)

2023-11-04 15:38:27,214 - BERTopic - Reduced dimensionality
2023-11-04 15:38:29,633 - BERTopic - Clustered reduced embeddings


In [28]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,1426,-1_patients_disease_covid19_clinical,"[patients, disease, covid19, clinical, treatme...",[Several viruses cause pulmonary infections du...
1,0,259,0_patients_covid19_clinical_virus,"[patients, covid19, clinical, virus, infection...",[BACKGROUND: Clinical trials applying iota-car...
2,1,233,1_covid19_patients_vaccine_clinical,"[covid19, patients, vaccine, clinical, disease...",[BACKGROUND: Emerging and epidemic infectious ...
3,2,196,2_patients_covid19_infection_human,"[patients, covid19, infection, human, vaccine,...",[Conventional vaccines have been extremely suc...
4,3,117,3_clinical_infection_patients_disease,"[clinical, infection, patients, disease, resea...",[BACKGROUND: The 2019 novel coronavirus diseas...
5,4,116,4_patients_covid19_clinical_severe,"[patients, covid19, clinical, severe, viral, r...",[INTRODUCTION: Dengue infection is the fastest...
6,5,96,5_clinical_covid19_studies_trials,"[clinical, covid19, studies, trials, patients,...",[BACKGROUND: Since the outbreak of coronavirus...
7,6,87,6_patients_virus_covid19_respiratory,"[patients, virus, covid19, respiratory, viruse...",[BACKGROUND: Community transmission of coronav...
8,7,78,7_respiratory_patients_viruses_disease,"[respiratory, patients, viruses, disease, vira...",[Antibody-based therapeutics currently enjoy u...
9,8,59,8_respiratory_viral_covid19_infections,"[respiratory, viral, covid19, infections, seve...",[Drug repurposing offers an expedited and econ...


In [29]:
topic_df = topic_model.get_topic_info()
topic_df[['Topic','Count']]

,Topic,Count
0,-1,1426
1,0,259
2,1,233
3,2,196
4,3,117
5,4,116
6,5,96
7,6,87
8,7,78
9,8,59


In [30]:
# Intertopic Distance Map
topic_model.visualize_topics(title='<b>PUBMED ELECTRA</b>')

In [31]:
# Hierarchical Clustering
topic_model.visualize_hierarchy(title="<b>PUBMED ELECTRA</b>")

In [32]:
abs_topics = pd.DataFrame({"topic": topics, "pmcid":df['pmcid'], "abstracts": df["abstract"].to_list()})

In [33]:
abs_topics

,topic,pmcid,abstracts
0,2,PMC137267,Recent evidence suggests that critically ill p...
1,-1,PMC1475929,The anticancer potential of catechins derived ...
2,-1,PMC1481583,BACKGROUND: Patients with chronic obstructive ...
3,5,PMC1525174,BACKGROUND: Because smallpox (variola major) m...
4,1,PMC1764036,BACKGROUND: The time delay between the start o...
...,...,...,...
3108,2,PMC5722503,Human cytomegalovirus (HCMV) utilizes RNA poly...
3109,2,PMC7339790,T cell reactivity against SARS-CoV-2 was obser...
3110,3,PMC7108131,The Ebola virus disease outbreak in west Afric...
3111,2,PMC4183573,Viruses utilize host factors for their efficie...


In [34]:
abs_topics.to_excel(
    "/home/maaz-lfd/Maaz/Thesis/Thesis/dataset/clinical_trials/topic_abstract_map_2.xlsx",
    index=False,
)